### Exploring Data

In [43]:
!pip install nltk

!pip install tensorflow

In [44]:
# Libraries for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Libraries for Tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#load the intents.json file from your local device
from google.colab import files
files.upload()

In [ ]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
intents

### Preprocessing The Data

In [ ]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
52 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


### Creating and Training the Model

In [ ]:
# creating training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of word for each sentencese
for doc in documents:
  # initialize bag of words
  bag = []
  # list of tokenized words for the patterns
  pattern_words = doc[0]
  # stemming each word
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  # create bag of words array
  for w in words:
    bag.append(1)  if w in pattern_words else bag.append(0)

  # output is '1' for current tag and '0' for rest of other tags
  output_row  = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])


<ipython-input-23-65f997850223>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]),activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=8, verbose=1)
model.save("model.pkl")

Epoch 1/100
4/4 [==============================] - 1s 6ms/step - loss: 2.2493 - accuracy: 0.0741
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 2.2181 - accuracy: 0.0741
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1908 - accuracy: 0.0741
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1683 - accuracy: 0.1111
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1426 - accuracy: 0.1481
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1205 - accuracy: 0.1481
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0997 - accuracy: 0.1852
Epoch 8/100
4/4 [==============================] - 0s 5ms/step - loss: 2.0767 - accuracy: 0.1852
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 2.0581 - accuracy: 0.1852
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0365 - accuracy: 0.2222
Epoch 11/100
4/4 [===========

### Making Prediction using Chatbot

In [ ]:
import pickle
pickle.dump({'words':words, 'classes':classes}, open("training_data","wb"))

In [ ]:
from keras.models import load_model
model = load_model('model.pkl')

In [ ]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [ ]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [ ]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
response('Where are you located?')

1/1 [==============================] - 0s 23ms/step
You can visit India to meet us


In [ ]:
response('That is helpful')

1/1 [==============================] - 0s 22ms/step
Any time!


In [ ]:
response('Bye')

1/1 [==============================] - 0s 24ms/step
Have a nice day


In [ ]:
response('Who are you?')

1/1 [==============================] - 0s 23ms/step
Hi, I am Mayank. Nice to meet you. I made this chatbot for fun and practice.
